# HST's Tranisting Exoplanet Noise Simulator 

This file demonstrates how to predict the:
    1. Transmission/emission spectrum S/N ratio
    2. Observation start window
for any system observed with WFC3/IR.

#### Background information
    Pandeia: ETC for JWST
    PandExo: Exoplanet noise simulator for JWST

In [1]:
import sys
sys.path.append('..')
import pandexo.engine.justdoit as jdi

/Users/stevekb1/opt/anaconda3/envs/pandexo-1p4/lib/python3.6/site-packages/pysynphot-0.9.14-py3.6-macosx-10.9-x86_64.egg/pysynphot/refs.py:118: UserWarning: No graph or component tables found; functionality will be SEVERELY crippled. No files found for /Users/stevekb1/Documents/data/pysynphot/cdbs/mtab/*_tmg.fits
  'functionality will be SEVERELY crippled. ' + str(e))
/Users/stevekb1/opt/anaconda3/envs/pandexo-1p4/lib/python3.6/site-packages/pysynphot-0.9.14-py3.6-macosx-10.9-x86_64.egg/pysynphot/refs.py:125: UserWarning: No thermal tables found, no thermal calculations can be performed. No files found for /Users/stevekb1/Documents/data/pysynphot/cdbs/mtab/*_tmt.fits
  'no thermal calculations can be performed. ' + str(e))


## Edit Inputs

### Load in a blank exoplanet dictionary with target name

In [2]:
exo_dict = jdi.load_exo_dict('WASP-43b')

In [3]:
exo_dict

{'calculation': 'fml',
 'star': {'type': 'phoenix',
  'starpath': 'filename_star',
  'w_unit': 'Angs,um,nm,Hz,sec,cm',
  'f_unit': 'FLAM or W/m2/um or erg/s/cm2/Hz or Jy',
  'mag': 9.995,
  'ref_wave': 1.25,
  'temp': 4400.0,
  'metal': -0.01,
  'logg': 4.5,
  'radius': 0.6,
  'r_unit': 'R_sun',
  'jmag': 9.995,
  'hmag': 9.397},
 'planet': {'type': 'user',
  'exopath': 'filename_planet',
  'w_unit': 'Angs,um,nm,Hz,sec,cm',
  'f_unit': 'rp^2/r*^2 or fp/r*',
  'transit_duration': 0.0483,
  'td_unit': 'd',
  'depth': 0.025370807084046897,
  'i': 82.149,
  'a': None,
  'period': 0.813475,
  'ecc': 0.0,
  'w': 90.0,
  'temp': None,
  'chem': None,
  'cloud': None,
  'radius': 0.93,
  'r_unit': 'R_jupiter',
  'mass': 1.78,
  'm_unit': 'M_jupiter',
  'ars': 5.089094350869629},
 'observation': {'sat_level': None,
  'sat_unit': '% full well or e',
  'noccultations': 1,
  'R': None,
  'baseline': 1,
  'baseline_unit': 'frac',
  'target_acq': False,
  'noise_floor': 0.0}}

#### Edit Planet Model

In [4]:
#WASP-43b
exo_dict['planet']['type']    = 'user'               # user specified inputs
exo_dict['planet']['exopath'] = jdi.os.getcwd()+'/WASP43b-Eclipse_Spectrum.txt' # filename for model spectrum
exo_dict['planet']['w_unit']  = 'um'                 # wavelength unit
exo_dict['planet']['f_unit']  = 'fp/f*'              # flux ratio unit (can also put "rp^2/r*^2")

### Step 2) Load in instrument dictionary
 
- WFC3 G141
- WFC3 G102

In [5]:
inst_dict = jdi.load_mode_dict('WFC3 G141')

#### Edit HST/WFC3 detector and observation inputs

In [6]:
exo_dict['observation']['noccultations']               = 5            # Number of transits/eclipses
inst_dict['configuration']['detector']['subarray']     = 'GRISM256'   # GRISM256 or GRISM512
inst_dict['configuration']['detector']['nsamp']        = 10           # WFC3 N_SAMP, 1..15
inst_dict['configuration']['detector']['samp_seq']     = 'SPARS10'    # WFC3 SAMP_SEQ, SPARS5 or SPARS10
inst_dict['strategy']['norbits']                       = 3            # Number of HST orbits
inst_dict['strategy']['nchan']                         = 15           # Number of spectrophotometric channels
inst_dict['strategy']['scanDirection']                 = 'Forward'    # Spatial scan direction, Forward or Round Trip
inst_dict['strategy']['schedulability']                = 30           # 30 for small/medium program, 100 for large program
inst_dict['strategy']['windowSize']                    = 20           # (optional) Observation start window size in minutes. Default is 20 minutes.
inst_dict['strategy']['useFirstOrbit']                 = True         # (optional) Default is False, option to use first orbit
inst_dict['strategy']['calculateRamp']                 = True         # Enables ramp effect simulation for flux plot
inst_dict['strategy']['targetFluence']                 = 30000        # Maximum pixel fluence level (in electrons)

## Run PandExo

`    jdi.run_pandexo(exo, inst, param_space = 0, param_range = 0,save_file = True,
                            output_path=os.getcwd(), output_file = '')`

See wiki Attributes for more thorough explanation fo inputs


In [7]:
foo = jdi.run_pandexo(exo_dict, inst_dict, output_file='wasp43b.p')
foo['wfc3_TExoNS']['info']

Running Single Case w/ User Instrument Dict
****WARNING: Number of specified HST orbits does not match number of recommended orbits: 4


{'Number of HST orbits': 3,
 'Use first orbit': True,
 'WFC3 parameters: NSAMP': 10,
 'WFC3 parameters: SAMP_SEQ': 'SPARS10',
 'Scan Direction': 'Forward',
 'Recommended scan rate (arcsec/s)': 0.0631662763955337,
 'Scan height (pixels)': 34.66131959252937,
 'Maximum pixel fluence (electrons)': 30000.0,
 'exposure time': 66.3965,
 'Estimated duty cycle (outside of Earth occultation)': 48.69076666666666,
 'Transit depth uncertainty(ppm)': 50.28677952013035,
 'Number of channels': 15,
 'Number of Transits': 5,
 'Start observations between orbital phases': '0.3890220624139921-0.40609559127337796'}

In [8]:
inst_dict['configuration']['detector']['nsamp'] = None
inst_dict['configuration']['detector']['samp_seq'] = None
bar = jdi.run_pandexo(exo_dict, inst_dict, output_file='wasp43b.p')
bar['wfc3_TExoNS']['info']

Running Single Case w/ User Instrument Dict
****WARNING: Number of specified HST orbits does not match number of recommended orbits: 4


{'Number of HST orbits': 3,
 'Use first orbit': True,
 'WFC3 parameters: NSAMP': 7,
 'WFC3 parameters: SAMP_SEQ': 'SPARS25',
 'Scan Direction': 'Forward',
 'Recommended scan rate (arcsec/s)': 0.0631662763955337,
 'Scan height (pixels)': 70.13753635409532,
 'Maximum pixel fluence (electrons)': 30000.0,
 'exposure time': 134.35399999999998,
 'Estimated duty cycle (outside of Earth occultation)': 58.22006666666667,
 'Transit depth uncertainty(ppm)': 45.987603738723394,
 'Number of channels': 15,
 'Number of Transits': 5,
 'Start observations between orbital phases': '0.3890220624139921-0.40609559127337796'}

In [9]:
inst_dict['strategy']['scanDirection'] = 'Round Trip'
hst = jdi.run_pandexo(exo_dict, inst_dict, output_file='wasp43b.p')
hst['wfc3_TExoNS']['info']

Running Single Case w/ User Instrument Dict
****WARNING: Number of specified HST orbits does not match number of recommended orbits: 4


{'Number of HST orbits': 3,
 'Use first orbit': True,
 'WFC3 parameters: NSAMP': 7,
 'WFC3 parameters: SAMP_SEQ': 'SPARS25',
 'Scan Direction': 'Round Trip',
 'Recommended scan rate (arcsec/s)': 0.0631662763955337,
 'Scan height (pixels)': 70.13753635409532,
 'Maximum pixel fluence (electrons)': 30000.0,
 'exposure time': 134.35399999999998,
 'Estimated duty cycle (outside of Earth occultation)': 71.65546666666667,
 'Transit depth uncertainty(ppm)': 41.45266582892166,
 'Number of channels': 15,
 'Number of Transits': 5,
 'Start observations between orbital phases': '0.3890220624139921-0.40609559127337796'}

## Plot Results


#### Plot simulated spectrum using specified file

In [10]:
import pandexo.engine.justplotit as jpi 
#using foo from above
#other keys include model=True/False
datawave, dataspec, dataerror, modelwave, modelspec = jpi.hst_spec(foo)

## Compute earliest and latest possible start times for given start window size

In [11]:
#using foo from above
obsphase1, obstr1, obsphase2, obstr2,rms = jpi.hst_time(foo)

# Compute simulated lightcurves in fluence (unit: electrons)
if configuration option `inst_dict['strategy']['calculateRamp']` is set as `True`, lightcurves will have ramp effect systematics

In [12]:
obsphase1, counts1, obsphase2, counts2, noise = jpi.hst_simulated_lightcurve(foo)

## Print important info for observation

In [13]:
hst['wfc3_TExoNS']['info']

{'Number of HST orbits': 3,
 'Use first orbit': True,
 'WFC3 parameters: NSAMP': 7,
 'WFC3 parameters: SAMP_SEQ': 'SPARS25',
 'Scan Direction': 'Round Trip',
 'Recommended scan rate (arcsec/s)': 0.0631662763955337,
 'Scan height (pixels)': 70.13753635409532,
 'Maximum pixel fluence (electrons)': 30000.0,
 'exposure time': 134.35399999999998,
 'Estimated duty cycle (outside of Earth occultation)': 71.65546666666667,
 'Transit depth uncertainty(ppm)': 41.45266582892166,
 'Number of channels': 15,
 'Number of Transits': 5,
 'Start observations between orbital phases': '0.3890220624139921-0.40609559127337796'}